In [ ]:
from google.colab import drive
drive.mount('/content/drive')     

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import json


def convert_file(input_file, output_file, tag2query_file):
    all_data = json.load(open(input_file))
    tag2query = json.load(open(tag2query_file))

    output = []
    origin_count = 0
    new_count = 0

    for data in all_data:
      for elem in data:
          origin_count += 1
          context = elem["context"]
          label2positions = elem["label"]
          for tag_idx, (tag, query) in enumerate(tag2query.items()):
              positions = label2positions.get(tag, [])
              mrc_sample = {
                  "context": context,
                  "query": query,
                  "start_position": [int(x.split(",")[0]) for x in positions],
                  "end_position": [int(x.split(",")[1]) for x in positions],
                  "qas_id": f"{origin_count}.{tag_idx}"
              }
              output.append(mrc_sample)
              new_count += 1

    json.dump(output, open(output_file, "w"), ensure_ascii=False, indent=2)
    print(f"Convert {origin_count} samples to {new_count} samples and save to {output_file}")

for file in ['dev', 'train', 'test']:
  input_file = f'/content/drive/MyDrive/Diploma_paper/NEREL_01/{file}.json'
  output_file = f'/content/drive/MyDrive/Diploma_paper/NEREL_MRC/mrc-ner.{file}'
  tag2query_file = f'/content/drive/MyDrive/Diploma_paper/mrc_preprocess_wanglaiki/mrc-wanglaiki/data_preprocess/queries/ent_nerel.json'
  convert_file(input_file, output_file, tag2query_file)


# def main():
#     genia_raw_dir = "/mnt/mrc/genia/genia_raw"
#     genia_mrc_dir = "/mnt/mrc/genia/genia_raw/mrc_format"
#     tag2query_file = "queries/genia.json"
#     os.makedirs(genia_mrc_dir, exist_ok=True)
#     for phase in ["train", "dev", "test"]:
#         old_file = os.path.join(genia_raw_dir, f"{phase}.genia.json")
#         new_file = os.path.join(genia_mrc_dir, f"mrc-ner.{phase}")
#         convert_file(old_file, new_file, tag2query_file)

Convert 1213 samples to 35177 samples and save to /content/drive/MyDrive/Diploma_paper/NEREL_MRC/mrc-ner.dev
Convert 9431 samples to 273499 samples and save to /content/drive/MyDrive/Diploma_paper/NEREL_MRC/mrc-ner.train
Convert 1288 samples to 37352 samples and save to /content/drive/MyDrive/Diploma_paper/NEREL_MRC/mrc-ner.test


In [ ]:
!pip install torch

In [ ]:

all_data = json.load(open('/content/drive/MyDrive/Diploma_paper/NEREL_MRC/shan_train.json', encoding="utf-8"))

item = 0 
def preprocess(item):
  data = all_data[item]

  #print(f'data {data}', flush=True)
  #print('Passed 0', flush=True)
  #print(f'tokenizer {tokenizer}', flush=True)
  qas_id = data.get("qas_id", "0.0")
  sample_idx, label_idx = qas_id.split(".")
  sample_idx = torch.LongTensor([int(sample_idx)])
  label_idx = torch.LongTensor([int(label_idx)])

  query = data["query"]
  context = data["context"]
  start_positions = data["start_position"]
  end_positions = data["end_position"]

  # if self.is_chinese:
  #     context = "".join(context.split())
  #     end_positions = [x+1 for x in end_positions]
  # else:
  # add space offsets
  words = context.split(' ')
  start_positions = [x + sum([len(w) for w in words[:x]]) for x in start_positions]
  end_positions = [x + sum([len(w) for w in words[:x + 1]]) for x in end_positions]

  query_context_tokens = tokenizer.encode(query, context, add_special_tokens=True)
  tokens = query_context_tokens.ids
  type_ids = query_context_tokens.type_ids
  offsets = query_context_tokens.offsets

  # find new start_positions/end_positions, considering
  # 1. we add query tokens at the beginning
  # 2. word-piece tokenize
  origin_offset2token_idx_start = {}
  origin_offset2token_idx_end = {}
  for token_idx in range(len(tokens)):
      # skip query tokens
      if type_ids[token_idx] == 0:
          continue
      token_start, token_end = offsets[token_idx]
      # skip [CLS] or [SEP]
      if token_start == token_end == 0:
          continue
      origin_offset2token_idx_start[token_start] = token_idx
      origin_offset2token_idx_end[token_end] = token_idx
  #print(f'origin_offset2token_idx_start {origin_offset2token_idx_start}', flush=True)
  #print(f'origin_offset2token_idx_end  {origin_offset2token_idx_end}', flush=True)
  #print(f'start_positions {start_positions} end_positions {end_positions}', flush=True)
  new_start_positions = [origin_offset2token_idx_start[start] for start in start_positions]
  #print(f'Passed 1', flush=True)
  new_end_positions = [origin_offset2token_idx_end[end] for end in end_positions]
  #print(f'Passed 2', flush=True)